# [DEV][PPO] Crawler

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import torch
import numpy as np

# widget bar to display progress
#!pip install progressbar
import progressbar as pb

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Crawler.app', worker_id=101,  no_graphics=True)
#env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: CrawlerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 129
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 20
        Vector Action descriptions: , , , , , , , , , , , , , , , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# name of brain
print('Name of brain:', brain_name)

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Name of brain: CrawlerBrain
Number of agents: 12
Size of each action: 20
There are 12 agents. Each observes a state with length: 129
The state for the first agent looks like: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.25000000e+00
  1.00000000e+00  0.00000000e+00  1.78813934e-07  0.00000000e+00
  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.06093168e-01 -1.42857209e-01 -6.06078804e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.33339906e+00 -1.42857209e-01
 -1.33341408e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [4]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
#env.close()

Total score (averaged over agents) this episode: 0.28983102766020846


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
def saveTrainedModel(agent, path):
    state_dicts = {'model': agent.model_local.state_dict()}
    torch.save(state_dicts, path)
    
def loadTrainedModel(agent, path):
    state_dicts = torch.load(path,map_location={'cuda:0': 'cpu'})

    agent.model_local.load_state_dict(state_dicts['model'])
    
    return agent

In [6]:
from PPO_agent import PPO_Agent
#import PPO_util 

model_dir = 'saved_models/'
model_name = 'unity_continuous_' + str(brain_name) + '_' + str(num_agents) + '_agents.pt'

agent = PPO_Agent(env, state_size, action_size, num_agents=num_agents, seed=1234)
#agent = loadTrainedModel(agent, model_dir+model_name)

current device:  cpu


In [ ]:
episode_max = 50000 # training loop max iterations
episode_reward = 0.0
mean_rewards = []
e = 0

widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]

#widget = ['Episode: ', pb.Counter(),'/',str(episode_max),'  ',  
#          'eps reward: ', str(np.mean(episode_reward)) ,'  ',
#          'Avg score (100e): ', str(mean_rewards[-100:]) ,'  ',
#          'actor gain: ', str(np.mean(agent.actor_gain)) ,'  ',
#          'critic loss: ', str(np.mean(agent.critic_loss)) ,'  ',
#           pb.ETA(), ' ', pb.Bar(marker=pb.RotatingMarker()), '  ' ]

timer = pb.ProgressBar(widgets=widget, maxval=episode_max).start()


while e < episode_max:

    # collect trajectories
    agent.step()
    episode_reward = agent.episodic_rewards
    
    # display some progress every 20 iterations
    if agent.is_training:

        # get the average reward of the parallel environments
        mean_rewards.append(np.mean(episode_reward))        
        
        if (e+1)%1==0 :
            print("Episode: {}   score: {:.2f}   Avg score (100e): {:.2f}   "
                  "actor gain: {:.2f}   critic loss: {:.2f}".format(e+1, np.mean(episode_reward),
                                                                    np.mean(mean_rewards[-100:]),
                                                                    np.mean(agent.actor_gain), 
                                                                    np.mean(agent.critic_loss)))
            
        if np.mean(mean_rewards[-100:]) >= 100:
            print("Average score over all agents across 100th episodes > 100. Problem Solved!")
            break
                
        timer.update(e)
        
        e += 1
    else:
        print('\rFetching experiences... {} '.format(len(agent.memory.memory)), end="")
        
    #update progress widget bar
    #timer.update(e+1)
    
timer.finish()


Prefetch completed. Training starts! 
Number of Agents:  12
Device:  cpu


Episode: 1   score: 1.61   Avg score (100e): 1.61   actor gain: -0.65   critic loss: 1.17


Episode: 2   score: 4.77   Avg score (100e): 3.19   actor gain: -0.65   critic loss: 0.91


Episode: 3   score: 6.67   Avg score (100e): 4.35   actor gain: -0.65   critic loss: 0.79


Episode: 4   score: 8.53   Avg score (100e): 5.39   actor gain: -0.65   critic loss: 0.71


...

Episode: 509   score: 26.45   Avg score (100e): 25.99   actor gain: -0.45   critic loss: 0.45


Episode: 510   score: 26.48   Avg score (100e): 26.00   actor gain: -0.45   critic loss: 0.45


Episode: 511   score: 26.49   Avg score (100e): 26.01   actor gain: -0.45   critic loss: 0.45


Episode: 512   score: 26.52   Avg score (100e): 26.02   actor gain: -0.45   critic loss: 0.45


Episode: 513   score: 26.50   Avg score (100e): 26.03   actor gain: -0.45   critic loss: 0.45


In [ ]:
saveTrainedModel(agent, model_dir + model_name)

In [ ]:
# plot the scores
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(mean_rewards)), mean_rewards)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
scores = np.zeros(num_agents)                # initialize the score (for each agent)
for _ in range(10):
    agent.step(train_mode=False)             # lower eps and train_mode=False
    episode_reward = agent.running_rewards
    scores += episode_reward                 # update the score (for each agent)
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()